In [1]:
import os
import imageio
import numpy as np
import cv2

In [17]:
def load_images(root):
    """load bmp images from root directory into np arrays"""
    
    paths = os.listdir(root)
    
    
    images = []
    
    for path in paths:
        if "bmp" not in path.split(".")[-1]:
            continue
        image = imageio.imread(os.path.join(root, path))
        images.append(np.array(image, dtype=np.uint8))
    
    return np.array(images, dtype=object)/255

In [18]:
def CCL(imgs):
    """gets np binary images and applies CCL. Returns labeled_img and number of labels"""
    
    
    labeled_imgs = []
    labels_sets = []
    
    for img in imgs:
        
        label_num = 1
        labels_set = []
        labeled_img = np.zeros(img.shape)
        #print(labeled_img.shape)
        
        
        for y in range(0, img.shape[0]):
            for x in range(0, img.shape[1]):
                if img[y][x] == 1:
                    
                    left_vote = 0
                    up_vote = 0
                    
                    if y > 1 and labeled_img[y-1][x] != 0:
                        up_vote = int(labeled_img[y-1][x])
                    if x > 1 and labeled_img[y][x-1] != 0:
                        left_vote = int(labeled_img[y][x-1])
                    
                    
                    if left_vote == 0 and up_vote!=0:
                        f_vote = up_vote
                    elif left_vote != 0 and up_vote==0:
                        f_vote = left_vote
                    else:
                        f_vote = min(left_vote, up_vote)
                    
                    if f_vote == 0:
                        labeled_img[y][x] = label_num
                        label_num += 1
                    else:
                        labeled_img[y][x] = f_vote
                    

                    if left_vote != 0 and up_vote != 0 and left_vote != up_vote:
                        labels_set.append([left_vote, up_vote])
        
        labels_set = merge_components(labels_set)
        
        for idx, x in enumerate(labels_set):
            for xx in x:
                min_val = min(x)
                #print(idx, xx)
                labeled_img = np.where(labeled_img == xx, idx+1, labeled_img)

        
        
        labeled_imgs.append(labeled_img)
        labels_sets.append(labels_set)

            
            
    return labeled_imgs, labels_sets

In [19]:
def merge_components(labels_set):
    """merges two sublists if they have a common element until there is no common elements between sublists"""
    
    
    result = []
    while len(labels_set)>0:

        first_element = labels_set[0]
        rest = labels_set[1:]
        
        len_first = -1
        while len(first_element)>len_first:
            
            
            len_first = len(first_element)
            
            rest_2 = []
            
            for x in rest:
                if bool(set(first_element) & set(x)):
                    first_element += x
                    first_element = list(set(first_element))
                else:
                    rest_2.append(x)
            
            rest = rest_2
            
        result.append(first_element.copy())
        labels_set = rest
        
    return result


In [53]:
def filter_size(thresh, binary_img):
    """Removes labels which are smaller than a the threshold size"""
    
    #binary_img = np.uint8(img)
    for x in np.unique(binary_img):
        if x == 0:
            continue
        indices = np.where(binary_img == x)
#         print(indices[0].shape)
#         print(indices[1].shape)
        if indices[0].shape[0] < thresh:
            binary_img[indices] = 0
            
            
    return binary_img

In [54]:
def color_mapper(label_num):
    
    if label_num == 0:
        return [122,0,67]
    elif label_num == 1:
        return [0,0,255]
    elif label_num == 2:
        return [0,255,0]
    elif label_num == 3:
        return [255,0,0]
    elif label_num == 4:
        return [255,255,0]
    elif label_num == 5:
        return [255,0,255]
    elif label_num == 6:
        return [56,32,78]
    elif label_num == 7:
        return [178,54,190]
    elif label_num == 8:
        return [12,54,122]
    elif label_num == 9:
        return [178,123,190]
    elif label_num == 10:
        return [100,23,90]
    
    

In [55]:
imgs = load_images("./data") #loading the images into np arrays
imgs.shape

(3,)

In [65]:
labeled_imgs, labels_sets = CCL(imgs)

for idx, img in enumerate(labeled_imgs):
    
    binary_img = np.uint8(img)
    
    if idx == 2: #apply size filtering on the second image
        binary_img = filter_size(300 ,binary_img)
    
    rgb_image = np.zeros((img.shape[0], img.shape[1], 3))
    
    for x in np.unique(binary_img):
        if x == 0:
            continue
        indices = np.where(binary_img == x)
        rgb_image[indices] = color_mapper(x)




    cv2.imwrite("output/out" + str(idx) + ".jpg" , rgb_image)
